# Previsões com as variáveis independentes para verificar se o dataset consegue prever a variável dependente.

In [2]:
import os
import numpy as np
import pandas as pd

def load_txt_files_from_folders(root_directory):
    relevant_features_data = {}

    # Verifica se o diretório raiz existe
    if not os.path.exists(root_directory):
        print(f"Diretório raiz não encontrado: {root_directory}")
        return relevant_features_data

    print(f"Iniciando a leitura do diretório: {root_directory}")

    # Percorre todas as subpastas dentro do diretório raiz
    for folder_name in os.listdir(root_directory):
        folder_path = os.path.join(root_directory, folder_name)

        if os.path.isdir(folder_path):
            print(f"Entrando na pasta: {folder_path}")
            txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

            if not txt_files:
                print(f"NENHUM arquivo .txt encontrado na pasta: {folder_name}")
                continue  # Pula para a próxima pasta se não houver arquivos .txt

            for filename in txt_files:
                file_path = os.path.join(folder_path, filename)
                print(f"\nProcessando arquivo: {file_path}")

                try:
                    with open(file_path, 'r') as file:
                        lines = file.readlines()

                        # Verifica se o arquivo está vazio
                        if len(lines) == 0:
                            print(f"Arquivo vazio: {file_path}")
                            continue

                        header = lines[0].strip()
                        data = [line.strip().split(',') for line in lines[1:] if line.strip()]

                        # Verifica se o conteúdo após o cabeçalho está vazio
                        if len(data) == 0:
                            print(f"Sem dados após o cabeçalho: {file_path}")
                            continue

                        # Conversão segura para float
                        try:
                            data = np.array(data, dtype=float)
                        except ValueError as e:
                            print(f"Erro ao converter dados em {file_path}: {e}")
                            print(f"Primeiras linhas problemáticas: {lines[1:5]}")  # Mostra as 5 primeiras linhas após o cabeçalho
                            continue

                        key = f"{folder_name}/{filename}"
                        relevant_features_data[key] = (header, data)
                        print(f"Arquivo carregado com sucesso: {key} - Shape: {data.shape}")

                except FileNotFoundError:
                    print(f"Arquivo não encontrado: {file_path}")
                except Exception as e:
                    print(f"Erro inesperado ao processar {file_path}: {e}")

    print("\nProcessamento concluído.")
    return relevant_features_data

# Exemplo de uso:
root_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem\saida\1-tratamento"
datasets = load_txt_files_from_folders(root_directory)

# Exibindo o resumo dos arquivos carregados
if datasets:
    print("\nResumo dos arquivos carregados:")
    for key, (header, data) in datasets.items():
        print(f"Arquivo processado: {key}")
        print(f"Header: {header}")
        print(f"Shape dos dados: {data.shape}")
        print("-" * 50)
else:
    print("Nenhum arquivo válido foi carregado.")


Iniciando a leitura do diretório: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem\saida\1-tratamento

Processamento concluído.
Nenhum arquivo válido foi carregado.


In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from itertools import product

# Função para realizar a previsão e avaliação usando KFold com diferentes hiperparâmetros
def predict_and_evaluate_kfold(data, header, n_splits=10, fit_intercept=True, positive=False):
    # Converter o array para um DataFrame
    df = pd.DataFrame(data, columns=header.split(','))

    # Separar variáveis independentes (X) e variável dependente (y)
    X = df.iloc[:, :-1].values  # Todas as colunas, exceto a última
    y = df.iloc[:, -1].values   # Última coluna (variável dependente)

    # Configurar KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    rmse_list = []
    r2_list = []

    # Treinar e testar o modelo para cada fold
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = LinearRegression(fit_intercept=fit_intercept, positive=positive)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calcular métricas para cada fold
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        rmse_list.append(rmse)
        r2_list.append(r2)

    # Retornar a média das métricas entre os folds
    return np.mean(rmse_list), np.mean(r2_list)

# Hiperparâmetros possíveis para LinearRegression
hyperparameters = {
    "fit_intercept": [True, False],
    "positive": [True, False]
}

# Aplicar o modelo para cada dataset
results = []
for key, (header, data) in datasets.items():
    print(f"\nProcessando arquivo: {key}")
    best_rmse, best_r2 = float('inf'), float('-inf')
    best_params = {}

    for params in product(*hyperparameters.values()):
        fit_intercept, positive = params

        for _ in range(30):  # Executa 30 vezes com diferentes divisões de KFold
            try:
                rmse, r2 = predict_and_evaluate_kfold(data, header, fit_intercept=fit_intercept, positive=positive)

                # Atualiza os melhores valores e hiperparâmetros
                if r2 > best_r2 or (r2 == best_r2 and rmse < best_rmse):
                    best_rmse, best_r2 = rmse, r2
                    best_params = {"fit_intercept": fit_intercept, "positive": positive}

            except Exception as e:
                print(f"Erro ao processar {key} na iteração {_ + 1} com {params}: {e}")

    # Armazena o melhor resultado para a base
    results.append((key, best_rmse, best_r2, best_params))

# Converter os resultados para um DataFrame
all_results_df = pd.DataFrame(results, columns=["Arquivo", "RMSE", "R²", "Melhores Hiperparâmetros"])

# Salvar o DataFrame em um arquivo Excel
output_path = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\1-tratamento\resultados.xlsx"
all_results_df.to_excel(output_path, index=False)
print(f"Todos os resultados foram salvos em '{output_path}'")


print("Todos os resultados foram salvos em 'resultados_hiperparametros.xlsx'")


Todos os resultados foram salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\1-tratamento\resultados.xlsx'
Todos os resultados foram salvos em 'resultados_hiperparametros.xlsx'
